In [272]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tweepy
import re
import emoji
import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())
stopword = nltk.corpus.stopwords.words('english')

import glob
import string
import math

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

[nltk_data] Downloading package words to C:\Users\SweetY
[nltk_data]     pAtel\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
movieInfoFile = 'data_files/u.item'
movieInfo = pd.read_csv(movieInfoFile, sep="|", header=None, index_col=False,
                  names=['itemId','title'], usecols=[0,1])
movieInfo

,itemId,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
...,...,...
1677,1678,Mat' i syn (1997)
1678,1679,B. Monkey (1998)
1679,1680,Sliding Doors (1998)
1680,1681,You So Crazy (1994)


In [3]:
movieInfo['year'] = movieInfo.title.str.extract('(\(\d\d\d\d\))',expand=False)
movieInfo['year'] = movieInfo.year.str.extract('(\d\d\d\d)',expand=False)

#Removing the years from the 'title' column
movieInfo['title'] = movieInfo.title.str.replace('(\(\d\d\d\d\))', '')

<ipython-input-3-2d0fe1d6a320>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  movieInfo['title'] = movieInfo.title.str.replace('(\(\d\d\d\d\))', '')


In [4]:
movieInfo

,itemId,title,year
0,1,Toy Story,1995
1,2,GoldenEye,1995
2,3,Four Rooms,1995
3,4,Get Shorty,1995
4,5,Copycat,1995
...,...,...,...
1677,1678,Mat' i syn,1997
1678,1679,B. Monkey,1998
1679,1680,Sliding Doors,1998
1680,1681,You So Crazy,1994


In [20]:
movieInfo['tagword'] = movieInfo['title'].str.lower()
movieInfo['tagword'] = movieInfo.tagword.str.replace(' ', '')
movieInfo['tagword'] = movieInfo['tagword'].apply(lambda x: "{}{}".format('#', x))
movieInfo['tagword'] = movieInfo.tagword.str.replace('(\([A-Za-z0-9_]+\))', '')
movieInfo

<ipython-input-20-657f3eff8acf>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  movieInfo['tagword'] = movieInfo.tagword.str.replace('(\([A-Za-z0-9_]+\))', '')


,itemId,title,year,tagword
0,1,Toy Story,1995,#toystory
1,2,GoldenEye,1995,#goldeneye
2,3,Four Rooms,1995,#fourrooms
3,4,Get Shorty,1995,#getshorty
4,5,Copycat,1995,#copycat
...,...,...,...,...
1677,1678,Mat' i syn,1997,#mat'isyn
1678,1679,B. Monkey,1998,#b.monkey
1679,1680,Sliding Doors,1998,#slidingdoors
1680,1681,You So Crazy,1994,#yousocrazy


In [193]:
movieInfo['title'][189]

'Henry V '

In [21]:
api_key = "0Nfp0QdlOHXy5FVWtI8T7xdct"
api_secret = "7IMXzjvfbStJejs1iI2B75esFcGehySRvWil31ZKGl48yNfr9E"
access_token = "1354846100847550467-Lelq383ZqoVZDlRTyj9BfmVkWNyWyU"
access_secret = "vqAMs6mi3oQpkwboh554koYc1e3ScEXbBdyGRqRV2E1Q5"

In [22]:
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth)

In [48]:
# function to perform data extraction
def scrape(movieID, movie_title, words):
    numtweet = 100  
    
    # Creating DataFrame using pandas
    db = pd.DataFrame(columns=['Movie Id', 'Title', 'Tagword', 'description', 'totaltweets', 'retweetcount', 'text', 'hashtags'])
      
    # We are using .Cursor() to search through twitter for the required tweets.
    # The number of tweets can be restricted using .items(number of tweets)
    tweets = tweepy.Cursor(api.search, q=words, lang="en",
                           tweet_mode='extended').items(numtweet)
     
    # .Cursor() returns an iterable object. Each item in 
    # the iterator has various attributes that you can access to 
    # get information about each tweet
    list_tweets = [tweet for tweet in tweets]
      
    # Counter to maintain Tweet Count
    i = 1  
      
    # we will iterate over each tweet in the list for extracting information about each tweet
    for tweet in list_tweets:
        movie_id = movieID
        title = movie_title
        tagword = words
        description = tweet.user.description
        totaltweets = tweet.user.statuses_count
        retweetcount = tweet.retweet_count
        hashtags = tweet.entities['hashtags']
          
        # Retweets can be distinguished by a retweeted_status attribute,
        # in case it is an invalid reference, except block will be executed
        try:
            text = tweet.retweeted_status.full_text
        except AttributeError:
            text = tweet.full_text
        hashtext = list()
        for j in range(0, len(hashtags)):
            hashtext.append(hashtags[j]['text'])
          
        # Here we are appending all the extracted information in the DataFrame
        ith_tweet = [movie_id, title, tagword, description,
                     totaltweets, retweetcount, text, hashtext]
        db.loc[len(db)] = ith_tweet
          
        # Function call to print tweet data on screen
#         print(ith_tweet)
        i = i+1
    filename = str(movie_title) + '_tweets.csv'
      
    # we will save our database as a CSV file.
    db.to_csv('movie_tweets_collected/' + filename)

In [ ]:
for index, row in movieInfo[189:].iterrows():
    movieID = row['itemId']
    movie_title = row['title']
    movie_tagword = row['tagword']
    scrape(movieID, movie_title, movie_tagword)

# Working with the collected Tweets.....

In [194]:
# specifying the path to csv files
path = "movie_tweets_collected"
  
# csv files in the path
files = glob.glob(path + "/*.csv")
  
print(len(files))
  
# defining an empty list to store 
# content
data_frame = pd.DataFrame()
content = []
  
# checking all the csv files in the 
# specified path
for filename in files:
    
    # reading content of csv file
#     content.append(filename)
    df = pd.read_csv(filename, index_col=None)
    content.append(df)

    
# converting content to data frame
data_frame = pd.concat(content)

174


In [195]:
data_frame

,Unnamed: 0,Movie Id,Title,Tagword,description,totaltweets,retweetcount,text,hashtags
0,0,178,12 Angry Men,#12angrymen,A podcast about the 1001 Movies You Must See B...,1180,0,All this and a joint recommendation for the “F...,"['12AngryMen', 'FilmTwitter']"
1,1,178,12 Angry Men,#12angrymen,@F1 @C4F1 @BTCC ⚒#WHUFC⚒ @WestHam #Columbo Fil...,249863,7,Who's looking after the farm while you're away...,[]
2,2,178,12 Angry Men,#12angrymen,NaN,65942,7,Who's looking after the farm while you're away...,[]
3,3,178,12 Angry Men,#12angrymen,NaN,2531,7,Who's looking after the farm while you're away...,[]
4,4,178,12 Angry Men,#12angrymen,Jesus is Coming Soon❤️🙌\n18 /2003\nThoughts.Em...,1195,0,12 Angry Men (1957)\nFascinating Movie.... A C...,"['12angrymen', 'sidneylumet', 'henryfonda']"
...,...,...,...,...,...,...,...,...,...
97,97,132,"Wizard of Oz, The","#wizardofoz,the",Content wife and mum; lover of musicals and th...,10357,3,If there’s one thing we love as much as musica...,[]
98,98,132,"Wizard of Oz, The","#wizardofoz,the","Musical Theatre, amdram, gender equality in th...",765,3,If there’s one thing we love as much as musica...,[]
99,99,132,"Wizard of Oz, The","#wizardofoz,the",Longest running #MusicalTheatre group #InRdg S...,2082,3,If there’s one thing we love as much as musica...,"['InternationalCatDay', 'MusicalTheatre', 'Wiz..."
0,0,169,"Wrong Trousers, The","#wrongtrousers,the",35 year old non-binary Northern monkey. Drink...,884,1,Thanks for joining me for Ambient Lego Stream ...,[]


In [276]:
def cleaner(tweet):
#     # Remove @ sign
#     tweet = re.sub("@[A-Za-z0-9]+","",tweet)
    
    # Remove http links
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet)
    
    # Join the sub strings generated from re.sub
#     tweet = " ".join(tweet.split())
    
    # Remove puncutations
    tweet  = "".join([char for char in tweet if char not in string.punctuation])
    tweet = re.sub('[0-9]+', '', tweet)
    
    # Remove Emojis
    tweet = ''.join(c for c in tweet if c not in emoji.UNICODE_EMOJI)
    regrex_pattern = re.compile(pattern = "["
                                u"\U0001F600-\U0001F64F"  # emoticons
                                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                u"\U00002702-\U000027B0"
                                u"\U000024C2-\U0001F251"
                                u"\U00002500-\U00002BEF"  # chinese char
                                u"\U00002702-\U000027B0"
                                u"\U00002702-\U000027B0"
                                u"\U000024C2-\U0001F251"
                                u"\U0001f926-\U0001f937"
                                u"\U00010000-\U0010ffff"
                                u"\u2640-\u2642"
                                u"\u2600-\u2B55"
                                u"\u200d"
                                u"\u23cf"
                                u"\u23e9"
                                u"\u231a"
                                u"\ufe0f"  # dingbats
                                u"\u3030"
                               "]+", flags = re.UNICODE)
    tweet = regrex_pattern.sub(r'',tweet)
    
    # Remove hashtag sign and keep the text
    tweet = tweet.replace("#", "").replace("_", " ")
    
    #tokenize
    tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) \
         if w.lower() in words or not w.isalpha())
    
    return tweet

In [207]:
data_frame

,Movie Id,Title,Tagword,description,cleaned_tweets
0,178,12 Angry Men,#12angrymen,A podcast about the 1001 Movies You Must See B...,A about the You Must See Before You Die nan In...
1,178,12 Angry Men,#12angrymen,@F1 @C4F1 @BTCC ⚒#WHUFC⚒ @WestHam #Columbo Fil...,Columbo Music Beer Allotment Armed Supporter j...
2,178,12 Angry Men,#12angrymen,NaN,nan just an average person I like to play vide...
3,178,12 Angry Men,#12angrymen,NaN,nan Weird Art Art to the Artist Broadway Rob a...
4,178,12 Angry Men,#12angrymen,Jesus is Coming Soon❤️🙌\n18 /2003\nThoughts.Em...,is Coming Soon nan Small team of Foundation an...
...,...,...,...,...,...
97,132,"Wizard of Oz, The","#wizardofoz,the",Content wife and mum; lover of musicals and th...,nan graphic artist video food thrift store hun...
98,132,"Wizard of Oz, The","#wizardofoz,the","Musical Theatre, amdram, gender equality in th...",Focus ​ on ​ ​ GOT ​ ​ ​ ​ Musical ran by Gues...
99,132,"Wizard of Oz, The","#wizardofoz,the",Longest running #MusicalTheatre group #InRdg S...,่ ี what the you know R Only Please Monster Fu...
0,169,"Wrong Trousers, The","#wrongtrousers,the",35 year old non-binary Northern monkey. Drink...,A about the You Must See Before You Die nan In...


In [161]:
data_frame.to_csv('data_files/cleaned_movie_tweets')

# Sentiment Analysis

In [278]:
movie_tweets = pd.read_csv('data_files/cleaned_movie_tweets')
movie_tweets.drop(['Unnamed: 0', 'Unnamed: 0.1', 'totaltweets', 'retweetcount', 'text', 'hashtags'], axis='columns', inplace=True)
movie_tweets

,Movie Id,Title,Tagword,description,cleaned_tweets
0,28,Apollo 13,#apollo13,I’m a theatre kid who loves space. Proud Democ...,I ’ m a who space Democratic Artist Painter Ar...
1,28,Apollo 13,#apollo13,"Poet, Storyteller... at least trying to tell one.",Poet Storyteller at least trying to tell one w...
2,28,Apollo 13,#apollo13,An open source bot here to support all women w...,An open source bot here to support all who cod...
3,28,Apollo 13,#apollo13,👩‍💻 Our mission is to inspire you to excel in ...,Our mission is to inspire you to excel in tech...
4,28,Apollo 13,#apollo13,Log On http://t.co/Jd5OkgIQ3e And Stay Updated...,Log On And Stay With Steven Da Latest Merchand...
...,...,...,...,...,...
3955,66,While You Were Sleeping,#whileyouweresleeping,Always contrary. No Dms please. I am Proficie...,Batman Forever Twitter Bot An community for of...
3956,66,While You Were Sleeping,#whileyouweresleeping,Do not lead me down this road unless you’re su...,to excellence in the art form Everything about...
3957,66,While You Were Sleeping,#whileyouweresleeping,"Woof, Woof, Grrrrr. 🐾🐺🍖🍗\nI eat Noms | random ...",All I do is tweet favorite and Oh and share my...
3958,66,While You Were Sleeping,#whileyouweresleeping,simple in all aspect of life,in Finance Author of paranormal suspense myste...


In [285]:
# init the sentiment analyzer
sia = SentimentIntensityAnalyzer()
sia_score_comp = []
sia_label = []

In [286]:
for tweet in movie_tweets['cleaned_tweets']:
    score = sia.polarity_scores(tweet)
    sia_score_comp.append(score['compound'])
    if score['compound'] > 0:
        sia_label.append("Positive")
    elif score['compound'] < 0:
        sia_label.append("Negative")
    elif score['compound'] == 0:
        sia_label.append("Neutral")
    


print(len(sia_label))

3960


In [287]:
movie_tweets['Sentiment_score'] = sia_score_comp
movie_tweets['Sentiment_label'] = sia_label

In [288]:
movie_tweets

,Movie Id,Title,Tagword,description,cleaned_tweets,Sentiment_score,Sentiment_label
0,28,Apollo 13,#apollo13,I’m a theatre kid who loves space. Proud Democ...,I ’ m a who space Democratic Artist Painter Ar...,0.9992,Positive
1,28,Apollo 13,#apollo13,"Poet, Storyteller... at least trying to tell one.",Poet Storyteller at least trying to tell one w...,0.9988,Positive
2,28,Apollo 13,#apollo13,An open source bot here to support all women w...,An open source bot here to support all who cod...,0.9983,Positive
3,28,Apollo 13,#apollo13,👩‍💻 Our mission is to inspire you to excel in ...,Our mission is to inspire you to excel in tech...,0.9991,Positive
4,28,Apollo 13,#apollo13,Log On http://t.co/Jd5OkgIQ3e And Stay Updated...,Log On And Stay With Steven Da Latest Merchand...,0.9989,Positive
...,...,...,...,...,...,...,...
3955,66,While You Were Sleeping,#whileyouweresleeping,Always contrary. No Dms please. I am Proficie...,Batman Forever Twitter Bot An community for of...,0.9974,Positive
3956,66,While You Were Sleeping,#whileyouweresleeping,Do not lead me down this road unless you’re su...,to excellence in the art form Everything about...,0.9845,Positive
3957,66,While You Were Sleeping,#whileyouweresleeping,"Woof, Woof, Grrrrr. 🐾🐺🍖🍗\nI eat Noms | random ...",All I do is tweet favorite and Oh and share my...,0.9980,Positive
3958,66,While You Were Sleeping,#whileyouweresleeping,simple in all aspect of life,in Finance Author of paranormal suspense myste...,0.9975,Positive


In [289]:
movie_tweets.to_csv('data_files/movie_tweets_with_SA1.csv')